# Title
[]()

In [38]:
# from pandas import json_normalize  
import tabula
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
# from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

import warnings
warnings.simplefilter("ignore", category=FutureWarning)

# Trouble shoot parsing of part 7

In [15]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() == 0:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass

                file_dfs.append(df_list[page_number])
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
        except:
            print('Unable to save outputs')

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() == 0:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass

                file_dfs.append(df_list[page_number])
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 2
start = 5
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 7
filename = f'F22-1445 Records_Part{part}.pdf'
dfs7, reference7 = parse_pdf_tolist(filename, pages='83-84', filename=None)
# dfs7, reference7 = parse_pdf(filenames, pages='1-6', filename=None)
# dfs4, reference4 = parse_pdf(filenames, pages='all', filename='pdfs_parsed_to_df_list')
# dfs, unique_cols = parse_pdf(filenames, pages='1-2', filename='pdfs_parsed_to_df_list')


Extracting the following files:
F22-1445 Records_Part7.pdf
number of columns: 24
	page 1...
		Current number of columns: 24 24
		24, 24
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 25 24
		24, 24
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part7.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed',
       'unknown_closure_type', 'open', 'days_since_last_case',
       'active_clusters', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0'],
      dtype='object')
	Removing s.22(1) from column 2
Error concatenating dataframes from 

In [17]:
print(len(dfs7))
dfs7[0].head()

4


,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closed_by_order,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate
0,1044,NHA,nknown,None,12/31/2021,1/28/2022,NaN,NaN,Manufacturing/In\rdustry,4.0,...,No,NaN,Unknow\rn,No,No,NaN,No,52-2021,NaN,NaN
1,1045,NHA,nknown,None,1/2/2022,1/27/2022,NaN,NaN,Shelter/SRO/Con\rgregate Housing,6.0,...,No,NaN,Unknow\rn,No,No,NaN,No,1-2022,NaN,NaN
2,1046,NHA,nknown,None,1/4/2022,NaN,NaN,NaN,Manufacturing/In\rdustry,NaN,...,No,NaN,Unknow\rn,No,No,NaN,Yes,1-2022,NaN,NaN
3,1047,IHA,elowna,None,1/5/2022,NaN,1/5/2022,NaN,Restaurant/Bar/L\rounge,4.0,...,Yes,1/2/2022,No,No,No,NaN,No,1-2022,1-2022,NaN
4,1048,IHA,amloops,None,1/5/2022,NaN,1/5/2022,1/3/2022,Office,4.0,...,No,NaN,No,No,No,NaN,No,1-2022,NaN,NaN


In [18]:
dfs7[1].head()

,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closed_by_order,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate
0,1044,NHA,nknown,None,12/31/2021,1/28/2022,NaN,NaN,Manufacturing/In\rdustry,4.0,...,No,NaN,Unknow\rn,No,No,NaN,No,52-2021,NaN,NaN
1,1045,NHA,nknown,None,1/2/2022,1/27/2022,NaN,NaN,Shelter/SRO/Con\rgregate Housing,6.0,...,No,NaN,Unknow\rn,No,No,NaN,No,1-2022,NaN,NaN
2,1046,NHA,nknown,None,1/4/2022,NaN,NaN,NaN,Manufacturing/In\rdustry,NaN,...,No,NaN,Unknow\rn,No,No,NaN,Yes,1-2022,NaN,NaN
3,1047,IHA,elowna,None,1/5/2022,NaN,1/5/2022,NaN,Restaurant/Bar/L\rounge,4.0,...,Yes,1/2/2022,No,No,No,NaN,No,1-2022,1-2022,NaN
4,1048,IHA,amloops,None,1/5/2022,NaN,1/5/2022,1/3/2022,Office,4.0,...,No,NaN,No,No,No,NaN,No,1-2022,NaN,NaN


In [19]:
dfs7[2].head()

,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,blank0
0,1068,IHA,NaN,West Kelowna,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,3,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,NaN
1,1069,IHA,NaN,Penticton,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,3,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,NaN
2,1070,IHA,NaN,Kelowna,1/18/2022,NaN,1/18/2022,1/22/2022,Office,4,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,NaN
3,1071,IHA,NaN,Vernon,1/18/2022,NaN,1/18/2022,1/18/2022,Office,5,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,NaN
4,1072,IHA,NaN,Kelowna,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,5,...,1/16/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,NaN


In [20]:
dfs7[2].head()

,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,blank0
0,1068,IHA,NaN,West Kelowna,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,3,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,NaN
1,1069,IHA,NaN,Penticton,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,3,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,NaN
2,1070,IHA,NaN,Kelowna,1/18/2022,NaN,1/18/2022,1/22/2022,Office,4,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,NaN
3,1071,IHA,NaN,Vernon,1/18/2022,NaN,1/18/2022,1/18/2022,Office,5,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,NaN
4,1072,IHA,NaN,Kelowna,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,5,...,1/16/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,NaN


In [32]:
n_docs = 2
start = 5
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'
dfs1, reference1 = parse_pdf(filename, pages='1-2', filename=None)

Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
		Current number of columns: 22 22
		22, 22
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 22 22
		22, 22
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
Error concatenating dataframes from F22-1445 Records_Part1.pdf


In [33]:
print(len(dfs1))
dfs1[0].head()

4


,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,workplace_status,closure_y_n,closed_by_order,voluntary_closure,closure_date,partially_cosed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate
0,1,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN
1,2,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN
2,3,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Closed,Yes,Yes,Unknown,4/4/2021,No,No,NaN,13-2021,14-2021
3,4,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Closed,Yes,Yes,Unknown,4/7/2021,No,No,NaN,13-2021,14-2021
4,5,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Closed,Yes,Yes,Unknown,4/5/2021,No,No,NaN,13-2021,14-2021


In [34]:
dfs1[1].head()

,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,workplace_status,closure_y_n,closed_by_order,voluntary_closure,closure_date,partially_cosed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate
0,1,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN
1,2,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN
2,3,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Closed,Yes,Yes,Unknown,4/4/2021,No,No,NaN,13-2021,14-2021
3,4,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Closed,Yes,Yes,Unknown,4/7/2021,No,No,NaN,13-2021,14-2021
4,5,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Closed,Yes,Yes,Unknown,4/5/2021,No,No,NaN,13-2021,14-2021


## Iteration 2
Fix errors

In [39]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)

def parse_pdf(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() == 0:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass

                file_dfs.append(df_list[page_number])
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 2
start = 5
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
dfs_i2, reference_i2 = parse_pdf(filenames, pages='1-2', filename=None)
# dfs4, reference4 = parse_pdf(filenames, pages='all', filename='pdfs_parsed_to_df_list')
# dfs, unique_cols = parse_pdf(filenames, pages='1-2', filename='pdfs_parsed_to_df_list')


Extracting the following files:
F22-1445 Records_Part5.pdf
number of columns: 24
	page 1...
		Current number of columns: 24 24
		24, 24
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 24 24
		24, 24
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)

	Done
F22-1445 Records_Part6.pdf
number of columns: 22
		No headers found; using headers from previous document: Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed',
       'unknown_closure_type', 'open', 'days_since_last_case',

## Iteration 3
Previous errors in iteration 1 were due to not having imported pandas

In [40]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() == 0:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass

                file_dfs.append(df_list[page_number])
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
        except:
            print('Unable to save outputs')

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() == 0:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass

                file_dfs.append(df_list[page_number])
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 2
start = 5
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 7
filename = f'F22-1445 Records_Part{part}.pdf'
dfs7_i3, reference7_i3 = parse_pdf_tolist(filename, pages='84', filename=None)
# dfs7, reference7 = parse_pdf(filenames, pages='1-6', filename=None)
# dfs4, reference4 = parse_pdf(filenames, pages='all', filename='pdfs_parsed_to_df_list')
# dfs, unique_cols = parse_pdf(filenames, pages='1-2', filename='pdfs_parsed_to_df_list')


Extracting the following files:
F22-1445 Records_Part7.pdf
number of columns: 25
	page 1...
		Current number of columns: 25 25
		25, 25
	Removing s.22(1) from column 2

	Done
Columns unique to partF22-1445 Records_Part7.pdf: {'closed_by_order', 'latest_reported_cases', 'file', 'cases_when_cluster_confirmed', 'earliest_reported_cases', 'date_cluster_confirmed_public_health', 'active_clusters', 'closure_y_n', 'city', 'workplace_status', 'HA', 'open', 'workplace_address', 'date_cluster_no_longer_active', 'partially_closed', 'index', 'unknown_closure_type', 'closure_date', 'total_cases', 'Unnamed: 1', 'cluster_closed_epidate', 'voluntary_closure', 'workplace_name', 'cluster_confirmed_epidate', 'days_since_last_case', 'workplace_classification'}


In [43]:
print(len(dfs7_i3))
dfs7_i3[0].tail()

1


,index,HA,workplace_name,workplace_address,Unnamed: 1,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,...,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,file
45,1088,IHA,NaN,NaN,1/25/2022,NaN,1/25/2022,NaN,School/Daycare,3,...,No,No,No,NaN,No,4-2022,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
46,1089,IHA,NaN,Castlegar,1/25/2022,NaN,1/25/2022,NaN,School/Daycare,6,...,No,No,No,NaN,No,4-2022,4-2022,NaN,NaN,F22-1445 Records_Part7.pdf
47,1090,IHA,NaN,Kamloops,1/26/2022,NaN,1/26/2022,NaN,School/Daycare,7,...,No,No,No,NaN,No,4-2022,4-2022,NaN,NaN,F22-1445 Records_Part7.pdf
48,1091,IHA,NaN,Kamloops,1/27/2022,NaN,1/27/2022,NaN,School/Daycare,4,...,No,No,No,NaN,No,4-2022,4-2022,NaN,NaN,F22-1445 Records_Part7.pdf
49,1092,IHA,NaN,Revelstoke,1/28/2022,NaN,1/28/2022,NaN,School/Daycare,5,...,No,No,No,NaN,No,4-2022,2-2022,NaN,NaN,F22-1445 Records_Part7.pdf


In [46]:
dfs7_i3[0]['workplace_address'].isna().sum() == len(dfs7_i3[0])

False

In [47]:
dfs7_i3[0]['workplace_name'].isna().sum() == len(dfs7_i3[0])

True

## iteration 4
Fixed issue that prevented some columns from being aligned in part7.pdf

In [50]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass

                file_dfs.append(df_list[page_number])
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
        except:
            print('Unable to save outputs')

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass

                file_dfs.append(df_list[page_number])
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 2
start = 5
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 7
filename = f'F22-1445 Records_Part{part}.pdf'
dfs7_i4, reference7_i4 = parse_pdf_tolist(filename, pages='84', filename=None)
# dfs7, reference7 = parse_pdf(filenames, pages='1-6', filename=None)
# dfs4, reference4 = parse_pdf(filenames, pages='all', filename='pdfs_parsed_to_df_list')
# dfs, unique_cols = parse_pdf(filenames, pages='1-2', filename='pdfs_parsed_to_df_list')


Extracting the following files:
F22-1445 Records_Part7.pdf
number of columns: 25
	page 1...
		Current number of columns: 25 25
		25, 25
	Removing s.22(1) from column 2
	Shifting data from column 3(workplace_address)

	Done
Columns unique to partF22-1445 Records_Part7.pdf: {'closed_by_order', 'latest_reported_cases', 'file', 'cases_when_cluster_confirmed', 'earliest_reported_cases', 'date_cluster_confirmed_public_health', 'active_clusters', 'closure_y_n', 'city', 'workplace_status', 'HA', 'open', 'workplace_address', 'date_cluster_no_longer_active', 'partially_closed', 'index', 'unknown_closure_type', 'closure_date', 'total_cases', 'Unnamed: 1', 'cluster_closed_epidate', 'voluntary_closure', 'workplace_name', 'cluster_confirmed_epidate', 'days_since_last_case', 'workplace_classification'}


In [51]:
dfs7_i4[0]

,index,HA,workplace_name,workplace_address,Unnamed: 1,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,...,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,file
0,1068,IHA,NaN,None,West Kelowna,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
1,1069,IHA,NaN,None,Penticton,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
2,1070,IHA,NaN,None,Kelowna,1/18/2022,NaN,1/18/2022,1/22/2022,Office,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
3,1071,IHA,NaN,None,Vernon,1/18/2022,NaN,1/18/2022,1/18/2022,Office,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
4,1072,IHA,NaN,None,Kelowna,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/16/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
5,1073,IHA,NaN,None,Cranbrook,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
6,1074,IHA,NaN,None,NaN,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
7,1075,NHA,NaN,None,Unknown,1/18/2022,NaN,NaN,NaN,Health Care\rFacility\r(Acute/LTC),...,NaN,Unknow\rn,No,No,NaN,Yes,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
8,1076,NHA,NaN,None,Unknown,1/18/2022,1/28/2022,NaN,NaN,Shelter/SRO/Con\rgregate Housing,...,NaN,Unknow\rn,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
9,1077,IHA,NaN,None,NaN,1/19/2022,NaN,1/19/2022,NaN,School/Daycare,...,1/18/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf


In [54]:
len(dfs7_i4)

1

In [55]:
dfs7_i4b, reference7_i4b = parse_pdf(filename, pages='84-85', filename=None)

Extracting the following files:
F22-1445 Records_Part7.pdf
number of columns: 25
	page 1...
		Current number of columns: 25 25
		25, 25
	Removing s.22(1) from column 2
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 24 25
		25, 25
		Dropping columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part7.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'Unnamed: 1',
       'city', 'date_cluster_confirmed_public_health',
       'date_cluster_no_longer_active', 'earliest_reported_cases',
       'latest_reported_cases', 'workplace_classification',
       'cases_when_cluster_confirmed', 'total_cases', 'workplace_status',
       'closure_y_n', 'closed_by_order', 'voluntary_closure', 'closure_date',
       'partially_closed', 'unknown_closure_type', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate'],
      dtype='object')
	Attempting to consolidate rows that were incorrectly split: 
	R

In [56]:
dfs7_i4b

,index,HA,workplace_name,workplace_address,Unnamed: 1,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,...,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,file
0,1068.0,IHA,NaN,None,West Kelowna,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
1,1069.0,IHA,NaN,None,Penticton,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
2,1070.0,IHA,NaN,None,Kelowna,1/18/2022,NaN,1/18/2022,1/22/2022,Office,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
3,1071.0,IHA,NaN,None,Vernon,1/18/2022,NaN,1/18/2022,1/18/2022,Office,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
4,1072.0,IHA,NaN,None,Kelowna,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/16/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
5,1073.0,IHA,NaN,None,Cranbrook,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
6,1074.0,IHA,NaN,None,NaN,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
7,1075.0,NHA,NaN,None,Unknown,1/18/2022,NaN,NaN,NaN,Health Care\rFacility\r(Acute/LTC),...,NaN,Unknow\rn,No,No,NaN,Yes,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
8,1076.0,NHA,NaN,None,Unknown,1/18/2022,1/28/2022,NaN,NaN,Shelter/SRO/Con\rgregate Housing,...,NaN,Unknow\rn,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
9,1077.0,IHA,NaN,None,NaN,1/19/2022,NaN,1/19/2022,NaN,School/Daycare,...,1/18/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf


In [57]:
dfs7_i4c, reference7_i4c = parse_pdf(filename, pages='1', filename=None)

Extracting the following files:
F22-1445 Records_Part7.pdf
number of columns: 24
	page 1...
		Current number of columns: 24 24
		24, 24
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)

	Done
Columns unique to partF22-1445 Records_Part7.pdf: {'closed_by_order', 'latest_reported_cases', 'file', 'cases_when_cluster_confirmed', 'earliest_reported_cases', 'date_cluster_confirmed_public_health', 'active_clusters', 'closure_y_n', 'city', 'workplace_status', 'HA', 'open', 'workplace_address', 'unknown_closure_type', 'partially_closed', 'index', 'date_cluster_no_longer_active', 'closure_date', 'total_cases', 'cluster_closed_epidate', 'voluntary_closure', 'workplace_name', 'cluster_confirmed_epidate', 'days_since_last_case', 'workplace_classification'}


In [58]:
dfs7_i4c

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,file
0,738,VCH,None,None,Vancouver,9/4/2021,NaN,9/4/2021,10/7/2021,Shelter/SRO/Congregate\rHousing,...,No,NaN,No,No,Yes,NaN,No,35-2021,NaN,F22-1445 Records_Part7.pdf
1,739,VCH,None,None,Vancouver,9/4/2021,NaN,9/4/2021,9/22/2021,Other,...,No,NaN,No,No,Yes,NaN,No,35-2021,NaN,F22-1445 Records_Part7.pdf
2,740,VCH,None,None,Vancouver,9/4/2021,NaN,9/4/2021,9/21/2021,Other,...,No,NaN,No,No,Yes,NaN,No,35-2021,NaN,F22-1445 Records_Part7.pdf
3,741,VCH,None,None,Vancouver,9/4/2021,NaN,9/4/2021,9/21/2021,Other,...,No,NaN,No,No,Yes,NaN,No,35-2021,NaN,F22-1445 Records_Part7.pdf
4,742,VCH,None,None,Vancouver,9/4/2021,NaN,9/4/2021,9/20/2021,Other,...,No,NaN,No,No,Yes,NaN,No,35-2021,NaN,F22-1445 Records_Part7.pdf
5,743,VIHA,None,None,Comox-Strath,9/6/2021,NaN,9/4/2021,9/8/2021,Other,...,No,NaN,Unknow\rn,No,Yes,118,No,36-2021,NaN,F22-1445 Records_Part7.pdf
6,744,NHA,None,None,Unknown,9/6/2021,10/5/2021,NaN,NaN,Health Care Facility\r(Acute/LTC),...,No,NaN,Unknow\rn,No,No,NaN,No,36-2021,NaN,F22-1445 Records_Part7.pdf
7,745,IHA,None,None,Grand Forks,9/7/2021,NaN,9/7/2021,9/15/2021,Manufacturing/Industry,...,Yes,9/4/2021,No,No,No,NaN,No,36-2021,35-2021,F22-1445 Records_Part7.pdf
8,746,IHA,None,None,Kamloops,9/7/2021,NaN,9/7/2021,9/16/2021,Manufacturing/Industry,...,Yes,9/9/2021,No,No,No,NaN,No,36-2021,36-2021,F22-1445 Records_Part7.pdf
9,747,IHA,None,None,Kamloops,9/7/2021,NaN,9/7/2021,9/7/2021,Restaurant/Bar/Lounge,...,No,NaN,No,No,No,NaN,No,36-2021,NaN,F22-1445 Records_Part7.pdf


In [62]:
n_docs = 2
start = 5
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 7
filename = f'F22-1445 Records_Part{part}.pdf'
dfsi4, referencei4 = parse_pdf_tolist(filenames, pages='84-85', filename=None)

Extracting the following files:
F22-1445 Records_Part5.pdf
number of columns: 24
	page 1...
		Current number of columns: 24 24
		24, 24
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 24 24
		24, 24
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)

	Done
F22-1445 Records_Part6.pdf
number of columns: 24
	page 1...
		Current number of columns: 24 24
		24, 24
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 24 24
		24, 24
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)

	Done
Columns unique to partF22-1445 Records_Part5.pdf: none
Columns unique to partF22-1445 Records_Part6.pdf: none


In [64]:
print(len(dfsi4))
dfsi4[0]

2


,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,file
0,495,IHA,None,None,liver,5/25/2021,NaN,5/25/2021,5/31/2021,Restaurant/Bar/L\rounge,...,Yes,5/22/2021,No,No,No,NaN,No,21-2021,20-2021,F22-1445 Records_Part5.pdf
1,496,FHA,None,None,HILLIWACK,5/25/2021,6/6/2021,NaN,NaN,Retail,...,Unknown,4/28/2021,Yes,No,Yes,NaN,No,21-2021,17-2021,F22-1445 Records_Part5.pdf
2,497,FHA,None,None,URREY,5/25/2021,6/8/2021,NaN,NaN,Retail,...,Unknown,NaN,No,No,Yes,NaN,No,21-2021,NaN,F22-1445 Records_Part5.pdf
3,498,NHA,None,None,nknown,5/25/2021,6/7/2021,NaN,NaN,Manufacturing/In\rdustry,...,Yes,5/19/2021,Unknown,No,No,NaN,No,21-2021,20-2021,F22-1445 Records_Part5.pdf
4,499,IHA,None,None,elowna,5/26/2021,NaN,5/26/2021,5/29/2021,Transportation,...,Yes,6/1/2021,No,No,No,NaN,No,21-2021,22-2021,F22-1445 Records_Part5.pdf
5,500,FHA,None,None,ORT\rOODY,5/26/2021,6/7/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Unknown,5/26/2021,No,No,Yes,NaN,No,21-2021,21-2021,F22-1445 Records_Part5.pdf
6,501,FHA,None,None,OQUITLAM,5/26/2021,6/7/2021,NaN,NaN,Office,...,Unknown,5/27/2021,No,No,Yes,NaN,No,21-2021,21-2021,F22-1445 Records_Part5.pdf
7,502,FHA,None,None,APLE RIDGE,5/26/2021,7/16/2021,NaN,NaN,Other,...,Unknown,5/26/2021,No,No,Yes,NaN,No,21-2021,21-2021,F22-1445 Records_Part5.pdf
8,503,VIHA,None,None,anaimo,5/26/2021,NaN,5/19/2021,5/26/2021,Retail,...,No,NaN,Unknown,No,Yes,167,No,21-2021,NaN,F22-1445 Records_Part5.pdf
9,504,IHA,None,None,elowna,5/27/2021,NaN,5/27/2021,6/3/2021,Restaurant/Bar/L\rounge,...,No,NaN,No,No,Yes,NaN,No,21-2021,NaN,F22-1445 Records_Part5.pdf


## Iteration 5: Final version of `parse_pdf_tolist` 
fix duplicate dataframes

In [65]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
        except:
            print('Unable to save outputs')

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass

            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 2
start = 5
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 7
filename = f'F22-1445 Records_Part{part}.pdf'
dfs7_i5, reference7_i5 = parse_pdf_tolist(filename, pages='84', filename=None)
# dfs7, reference7 = parse_pdf(filenames, pages='1-6', filename=None)
# dfs4, reference4 = parse_pdf(filenames, pages='all', filename='pdfs_parsed_to_df_list')
# dfs, unique_cols = parse_pdf(filenames, pages='1-2', filename='pdfs_parsed_to_df_list')


Extracting the following files:
F22-1445 Records_Part7.pdf
number of columns: 25
	page 1...
		Current number of columns: 25 25
		25, 25
	Removing s.22(1) from column 2
	Shifting data from column 3(workplace_address)

	Done
Columns unique to partF22-1445 Records_Part7.pdf: {'closed_by_order', 'latest_reported_cases', 'file', 'cases_when_cluster_confirmed', 'earliest_reported_cases', 'date_cluster_confirmed_public_health', 'active_clusters', 'closure_y_n', 'city', 'workplace_status', 'HA', 'open', 'workplace_address', 'date_cluster_no_longer_active', 'partially_closed', 'index', 'unknown_closure_type', 'closure_date', 'total_cases', 'Unnamed: 1', 'cluster_closed_epidate', 'voluntary_closure', 'workplace_name', 'cluster_confirmed_epidate', 'days_since_last_case', 'workplace_classification'}


In [66]:
print(len(dfs7_i5))
dfs7_i5[0]

1


,index,HA,workplace_name,workplace_address,Unnamed: 1,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,...,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,file
0,1068,IHA,NaN,None,West Kelowna,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
1,1069,IHA,NaN,None,Penticton,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
2,1070,IHA,NaN,None,Kelowna,1/18/2022,NaN,1/18/2022,1/22/2022,Office,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
3,1071,IHA,NaN,None,Vernon,1/18/2022,NaN,1/18/2022,1/18/2022,Office,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
4,1072,IHA,NaN,None,Kelowna,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/16/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
5,1073,IHA,NaN,None,Cranbrook,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
6,1074,IHA,NaN,None,NaN,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
7,1075,NHA,NaN,None,Unknown,1/18/2022,NaN,NaN,NaN,Health Care\rFacility\r(Acute/LTC),...,NaN,Unknow\rn,No,No,NaN,Yes,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
8,1076,NHA,NaN,None,Unknown,1/18/2022,1/28/2022,NaN,NaN,Shelter/SRO/Con\rgregate Housing,...,NaN,Unknow\rn,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
9,1077,IHA,NaN,None,NaN,1/19/2022,NaN,1/19/2022,NaN,School/Daycare,...,1/18/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf


In [67]:
dfs7_i5b, reference7_i5b = parse_pdf(filename, pages='84', filename=None)

Extracting the following files:
F22-1445 Records_Part7.pdf
number of columns: 25
	page 1...
		Current number of columns: 25 25
		25, 25
	Removing s.22(1) from column 2
	Shifting data from column 3(workplace_address)

	Done
Columns unique to partF22-1445 Records_Part7.pdf: {'closed_by_order', 'latest_reported_cases', 'file', 'cases_when_cluster_confirmed', 'earliest_reported_cases', 'date_cluster_confirmed_public_health', 'active_clusters', 'closure_y_n', 'city', 'workplace_status', 'HA', 'open', 'workplace_address', 'date_cluster_no_longer_active', 'partially_closed', 'index', 'unknown_closure_type', 'closure_date', 'total_cases', 'Unnamed: 1', 'cluster_closed_epidate', 'voluntary_closure', 'workplace_name', 'cluster_confirmed_epidate', 'days_since_last_case', 'workplace_classification'}


In [69]:
print(len(dfs7_i5b))
dfs7_i5b

25


,index,HA,workplace_name,workplace_address,Unnamed: 1,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,...,voluntary_closure,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,file
0,1068,IHA,NaN,None,West Kelowna,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
1,1069,IHA,NaN,None,Penticton,1/17/2022,NaN,1/17/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
2,1070,IHA,NaN,None,Kelowna,1/18/2022,NaN,1/18/2022,1/22/2022,Office,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
3,1071,IHA,NaN,None,Vernon,1/18/2022,NaN,1/18/2022,1/18/2022,Office,...,NaN,No,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
4,1072,IHA,NaN,None,Kelowna,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/16/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
5,1073,IHA,NaN,None,Cranbrook,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
6,1074,IHA,NaN,None,NaN,1/18/2022,NaN,1/18/2022,NaN,School/Daycare,...,1/17/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf
7,1075,NHA,NaN,None,Unknown,1/18/2022,NaN,NaN,NaN,Health Care\rFacility\r(Acute/LTC),...,NaN,Unknow\rn,No,No,NaN,Yes,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
8,1076,NHA,NaN,None,Unknown,1/18/2022,1/28/2022,NaN,NaN,Shelter/SRO/Con\rgregate Housing,...,NaN,Unknow\rn,No,No,NaN,No,3-2022,NaN,NaN,F22-1445 Records_Part7.pdf
9,1077,IHA,NaN,None,NaN,1/19/2022,NaN,1/19/2022,NaN,School/Daycare,...,1/18/2022,No,No,No,NaN,No,3-2022,3-2022,NaN,F22-1445 Records_Part7.pdf


# Fix column alignment in case `value_to_ignore` gets truncated during parsing
## Iteration 6

In [174]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)




def parse_pdf(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)', value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            # print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            # print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()

                for index, item in enumerate(row_values):
                    # print(index, item)
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break
                # value_to_ignore_regex = '.*' + value_to_ignore + '.*'
                # value_to_ignore_column = [index for index, item in enumerate(row_values) if re.search(value_to_ignore_regex, item)]

                # value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                
                # value_to_ignore_column = df_list[page_number][df_list[page_number].iloc[0].str.contains(
                #         value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index], regex=False, na=False,
                #         ) == True]
                    
                # print('value_to_ignore_column:', value_to_ignore_column)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 2
start = 5
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'
dfs1_i6, reference1_i6 = parse_pdf(filename, pages='10', filename=None)
# dfs7, reference7 = parse_pdf(filenames, pages='1-6', filename=None)
# dfs4, reference4 = parse_pdf(filenames, pages='all', filename='pdfs_parsed_to_df_list')
# dfs, unique_cols = parse_pdf(filenames, pages='1-2', filename='pdfs_parsed_to_df_list')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
	Removing s.22(1) from column 3
	Shifting data from column 3(workplace_address)

	Done
Columns unique to partF22-1445 Records_Part1.pdf: {'closed_by_order', 'latest_reported_cases', 'file', 'cases_when_cluster_confirmed', 'earliest_reported_cases', 'date_cluster_confirmed_public_health', 'closure_y_n', 'city', 'workplace_status', 'HA', 'open', 'workplace_address', 'date_cluster_no_longer_active', 'partially_closed', 'index', 'closure_date', 'total_cases', 'doc_page_number', 'cluster_closed_epidate', 'voluntary_closure', 'workplace_name', 'cluster_confirmed_epidate', 'days_since_last_case', 'workplace_classification'}


In [177]:
dfs1_i6.head()

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closed_by_order,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,file
0,150,FHA,NaN,None,BBOTSFORD,4/14/2021,5/3/2021,NaN,NaN,Farm/Agriculture,...,Yes,Unknown,4/15/2021,No,No,NaN,15-2021,15-2021,1,F22-1445 Records_Part1.pdf
1,151,FHA,NaN,None,URNABY,4/14/2021,4/27/2021,NaN,NaN,Transportation,...,No,Unknown,NaN,No,No,NaN,15-2021,NaN,1,F22-1445 Records_Part1.pdf
2,152,FHA,NaN,None,ELTA,4/14/2021,5/19/2021,NaN,NaN,Warehouse,...,No,Unknown,NaN,No,No,NaN,15-2021,NaN,1,F22-1445 Records_Part1.pdf
3,153,FHA,NaN,None,ELTA,4/14/2021,5/9/2021,NaN,NaN,Retail,...,No,Unknown,NaN,No,No,NaN,15-2021,NaN,1,F22-1445 Records_Part1.pdf
4,154,FHA,NaN,None,ORT\rOQUITLAM,4/14/2021,5/6/2021,NaN,NaN,Maintenance/Tr\rades/Constructio\rn,...,Yes,Unknown,4/14/2021,No,No,NaN,15-2021,15-2021,1,F22-1445 Records_Part1.pdf


In [178]:
reference1_i6

{'corrected pages': {},
 'unique columns': {'F22-1445 Records_Part1.pdf': {'HA',
   'cases_when_cluster_confirmed',
   'city',
   'closed_by_order',
   'closure_date',
   'closure_y_n',
   'cluster_closed_epidate',
   'cluster_confirmed_epidate',
   'date_cluster_confirmed_public_health',
   'date_cluster_no_longer_active',
   'days_since_last_case',
   'doc_page_number',
   'earliest_reported_cases',
   'file',
   'index',
   'latest_reported_cases',
   'open',
   'partially_closed',
   'total_cases',
   'voluntary_closure',
   'workplace_address',
   'workplace_classification',
   'workplace_name',
   'workplace_status'}}}

## iteration 7
Save CSV

In [180]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)




def parse_pdf(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)', value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            # print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            # print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(dfs_list, filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 3...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 4...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 5...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 6...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 7...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 8...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifti

In [181]:
dfs

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,blank0,blank1,blank2,file,active_clusters,unknown_closure_type
0,1,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,NaN,13-2021,NaN,1,NaN,NaN,NaN,F22-1445 Records_Part1.pdf,NaN,NaN
1,2,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,NaN,13-2021,NaN,1,NaN,NaN,NaN,F22-1445 Records_Part1.pdf,NaN,NaN
2,3,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,NaN,13-2021,14-2021,1,NaN,NaN,NaN,F22-1445 Records_Part1.pdf,NaN,NaN
3,4,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,NaN,13-2021,14-2021,1,NaN,NaN,NaN,F22-1445 Records_Part1.pdf,NaN,NaN
4,5,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,NaN,13-2021,14-2021,1,NaN,NaN,NaN,F22-1445 Records_Part1.pdf,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,1090.0,IHA,None,None,Kamloops,1/26/2022,NaN,1/26/2022,NaN,School/Daycare,...,NaN,4-2022,4-2022,81,NaN,NaN,NaN,F22-1445 Records_Part7.pdf,No,No
1398,1091.0,IHA,None,None,Kamloops,1/27/2022,NaN,1/27/2022,NaN,School/Daycare,...,NaN,4-2022,4-2022,81,NaN,NaN,NaN,F22-1445 Records_Part7.pdf,No,No
1399,1092.0,IHA,None,None,Revelstoke,1/28/2022,NaN,1/28/2022,NaN,School/Daycare,...,NaN,4-2022,2-2022,81,NaN,NaN,NaN,F22-1445 Records_Part7.pdf,No,No
1400,1093.0,IHA,None,None,260 Cedar Ave,None,1/31/2022,None,1/31/2022,None,...,No,No,5-2022,82,NaN,NaN,NaN,F22-1445 Records_Part7.pdf,None,No


In [182]:
filename='pdfs_parsed_2023-03-30'
path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'
save_csv(dfs, filename=filename, path=path)

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/BC-Covid-workplace-closures/data/interim/pdfs_parsed_2023-03-30.csv
Time completed: 2023-03-31 01:33:22.959314


In [184]:
with open(f'..data/interim/{filename}_reference.json','w') as json_file:
    json.dump(reference, json_file)

FileNotFoundError: [Errno 2] No such file or directory: '..data/interim/pdfs_parsed_2023-03-30_reference.json'

In [185]:
with open(f'{filename}_reference.json','w') as json_file:
    json.dump(reference, json_file)

# *End of Page*